In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
#first = input layer, last = output layer, all remainings = hidden layers
allLayers = {1:[1,2,3], 2:[4, 5], 3: [6,7], 4:[8,9,10], 5:[11]}
#allLayers = {1:[1,2,3], 2:[4, 5], 3: [6]}

In [3]:
import random
#generates random weight for each node specified in multi layer data structure
def randomWeightAndBiosGeneratorForNodes(layers):
  w = {}
  b = {}
  if(len(layers) <= 1):
    return w
  for i in range(len(layers)):
    current = layers[i+1]
    next = {}
    if(i+2 in layers):
      next = layers[i+2]
    for curr in current:
      b[curr] = round(random.uniform(1, 100), 2)
      for nxt in next:
        index =  int(str(curr) + str(nxt)) #link between two nodes
        w[index] = round(random.uniform(1, 100), 2)
  return w, b
weight, bios = randomWeightAndBiosGeneratorForNodes(allLayers)

#generates random features(/input) for input layer specified in multi layer data structure
def randomFeatureGenerator(inputLayer):
  x = {}
  for i in inputLayer:
    x[i] = round(random.uniform(1, 100), 2)
  return x
x = randomFeatureGenerator(allLayers[1])

In [ ]:
print("weight", weight)
print("bios:", bios)
print("x:", x)

weight {14: 27.02, 15: 81.41, 24: 10.26, 25: 10.04, 34: 58.02, 35: 20.82, 46: 78.15, 47: 67.86, 56: 68.01, 57: 86.87, 68: 93.19, 69: 13.02, 610: 54.98, 78: 30.8, 79: 26.61, 710: 64.64, 811: 16.81, 911: 2.21, 1011: 94.63}
bios: {1: 46.96, 2: 83.94, 3: 48.31, 4: 80.51, 5: 73.35, 6: 73.96, 7: 59.47, 8: 82.04, 9: 66.32, 10: 24.08, 11: 10.28}
x: {1: 72.75, 2: 47.88, 3: 96.28}


In [4]:
#calcuates z value (part of forward propogration) for given node, weight, features/x values and bios
def forwardProp(node, w, f, b):
  z = b
  print("\n---node:", node)
  for i in range(len(w)):
    print("w:", w[i], " x:", f[i], " bios/theta:", b)
    z = z + (w[i] * f[i])
  print("z is ", z)
  return z

#calcuates sigmoid for z value
def sigmoid(z):
    return (1/(1 + math.exp(-z)))

def calculateZandA(layers, w, b, x):
  z = {}
  a = {}
  nextX = x
  for i in range(len(layers)):
    current = layers[i+1]
    tempX = {}
    next = {}
    if(i+2 in layers):
      next = layers[i+2]
    for nxt in next:
      nxtNodeWeight = []
      for curr in current:
        index =  int(str(curr) + str(nxt)) #link between two nodes
        if index in w:
          nxtNodeWeight.append(w[index])
      z[nxt] = forwardProp(nxt, nxtNodeWeight, list(nextX.values()), bios[nxt])
      a[nxt] = sigmoid(z[nxt])
      tempX[nxt] = a[nxt]
    #set features/x for next layer
    nextX = tempX
  return z, a

z,a = calculateZandA(allLayers, weight, bios, x)
print("\n\nfinal result:")
print("z:", z)
print("a:", a)


---node: 4
w: 89.51  x: 74.1  bios/theta: 24.45
w: 94.22  x: 68.48  bios/theta: 24.45
w: 22.2  x: 74.05  bios/theta: 24.45
z is  14753.2366

---node: 5
w: 21.05  x: 74.1  bios/theta: 77.57
w: 96.47  x: 68.48  bios/theta: 77.57
w: 57.17  x: 74.05  bios/theta: 77.57
z is  12477.0791

---node: 6
w: 97.4  x: 1.0  bios/theta: 22.57
w: 2.64  x: 1.0  bios/theta: 22.57
z is  122.61

---node: 7
w: 72.73  x: 1.0  bios/theta: 49.88
w: 61.83  x: 1.0  bios/theta: 49.88
z is  184.44

---node: 8
w: 35.91  x: 1.0  bios/theta: 14.27
w: 49.3  x: 1.0  bios/theta: 14.27
z is  99.47999999999999

---node: 9
w: 88.06  x: 1.0  bios/theta: 42.23
w: 15.38  x: 1.0  bios/theta: 42.23
z is  145.67

---node: 10
w: 94.94  x: 1.0  bios/theta: 31.36
w: 13.4  x: 1.0  bios/theta: 31.36
z is  139.7

---node: 11
w: 29.85  x: 1.0  bios/theta: 19.7
w: 62.18  x: 1.0  bios/theta: 19.7
w: 60.98  x: 1.0  bios/theta: 19.7
z is  172.70999999999998


final result:
z: {4: 14753.2366, 5: 12477.0791, 6: 122.61, 7: 184.44, 8: 99.4799

In [ ]:
print("weight", weight)
print("bios:", bios)
print("x:", x)
#learning_rate = .9 #eta
#allLayers = {1:{1,2,3}, 2:{4, 5}, 3: {6}}
#x = {1:1, 2:0, 3:1}
#y = 1
#weight = {14:0.2, 15:-0.3, 24:0.4, 25:0.1, 34:-0.5, 35:0.2, 46:-0.3, 56:-0.2}
#bios = {4:-0.4, 5: 0.2, 6:0.1} #theta
#learning_rate = .9 #eta
#t = 1